# Schedule API Jobs

I have finally figured out how to **schedule** jobs that use the Webui API to train Loras, and I will go over the required techniques in this notebook.

## Environment Variables

In module 3.5 we worked on editing the .bashrc files to set the environment variable WEBUI_PORT that is used in some bash scripts that interact with the API.

When I wrote module 3.5 I thought that all users had their .local/bin folder in their paths, but that is not true. I had it in my path because of some installation that I had made.

We need another change to the .bashrc files to put those folders in your paths.

Here are the last few lines of the file .bashrc in my home directory:

<img src='../data/api-both.png'>

All of you need the two export lines in your .bashrc files.  Put your webui port number in where I have 7101.

In place of \\${USER}, you can just use your login name.  So, I could have used brownj instead of \${USER}.

After adding those lines to your .bashrc file the command `source .bashrc` will set those environment variables in your current shell.  When you login in the future the .bashrc is sourced automatically, so you will not need that command.

Now if you copy the scripts ping.sh, start.sh, and train_lora.sh in /opt/models/scripts folder to your .local/bin folders, you will be able to execute those scripts.

Here are the commands to copy the scripts:

**Commands**

>cd .local/bin
>
>cp /opt/models/scripts/\*.sh .

Remember to include the last dot.

**Note:** The next five sections are copied from module 3.5.

## ping.sh

This should return `API is available` if your webui API is running, and `API needs to be started` if it is not.

## start.sh

<img src='../data/api-start.png'>

* start.sh 3 starts the webui.sh with flags --api and --device-id=3
* First call to ping said the API needs to be started. It was still starting up.

Here is the script:

In [1]:
!cat ../data/start.sh

#!/bin/bash

cd /home/${USER}/stable-diffusion-webui

if [ $# -gt 0 ]
then
	nohup ./webui.sh --api --device-id=$1 &
else
	nohup ./webui.sh --api &
fi

echo $!>process_id


The command start.sh with no number after it starts webui without the device-id flag set, so you will be put on GPU 0.

## train_lora.sh

<img src='../data/api-train.png'>

Here is the script:

In [4]:
!cat ../data/train_lora.sh

#!/bin/bash

if [ $# -lt 1 ]
then
	echo "Supply your model name as an argument"
else
	myping="`curl -X 'GET' --silent --output /dev/null \
		--write-out '%{http_code}' \
		http://localhost:${WEBUI_PORT}/internal/ping`"

	if [ $myping -eq 200 ]
	then
		echo "Calling curl ..."
		url="http://localhost:${WEBUI_PORT}/dreambooth/start_training?model_name=${1}&use_tx2img=true"
        	echo $url

		resp="`curl -X 'POST' --silent $url -H 'accept: application/json' -d ''`"
		echo "Done with curl"
		echo $resp
	else
		echo "API needs to be started"
	fi
fi


## nohup.out

The start.sh script changed to the stable-diffusion directory and started the webui with nohup.

The output of the command is sent to the file nohup.out.

tail -f nohup.out will show the nohup.out file, updating as the file changes.

<img src='../data/api-tail.png'>

## Stop Webui

<img src='../data/api-stop.png'>

ps -ef shows details of running processes.

Pipe the output to grep and look for rows that contain launch.py.  The first one is the process we want to kill.

## Cron and Crontab

Cron is a scheduling utility for linux machines, and the scheduled jobs are stored in files called crontabs.

The command `crontab` allows you to view and edit your crontab files, and the command requires a flag.  The two most common flags are -l and -e:

>crontab -l

This command lists your crontab file.  If you have not scheduled any jobs, the output is `no crontab for user-name`.

>crontab -e

This command is for editing your crontab. It gives you a choice of editors, and it recommends nano as the simplest.

When you start editing the crontab file has quite a few lines of comments that start with #.  In this notebook I will show you an image of my crontab file, and I didn't want to include all the comments, so I edited all but the last one out.

The comment I left in the file is this:

>\# m h  dom mon dow   command

There are five numbers of sets/ranges of numbers followed by the command to execute.

m: minute

h: hour

dom: day of month

mon: month

dow: day of week

The reason for 'day of week' is that you might want a job run Mon-Fri, but not on the weekend. The training jobs we will be scheduling are one time events, so dow will be a \*.

**Example** Here is the crontab for the root user on dive10:

>0 4 * * * rsync -a /home /mnt/backup

The 0 4 mean minute zero of hour 4, or 4:00 am.

The three stars mean every day, every month, and every day of the week.

The command is to have rsync backup the directory /home to the directory /mnt/backup.  So, every user's home directory is backed up to a second hard drive every day at 4:00 am.

**Important Fact:**

>When the cron daemon executes a job for you, **it does not have your environment variables**.
>
>This means that it does not have your path, so you need to use the complete path to most commands. The rsync command is on the very limited path available to the cron daemon, so in the example above the complete path to the rsync command is not needed.
>
>It also means that your environment variables, such as WEBUI_PORT, are not known, so if the command you want to execute needs that variable, and train_lora.sh does need it, then you have to supply it.

Here is a screen shot of my crontab file that I used to train a lora named foobar:

<img src='../data/cron-tab.png'>

**Notes:**

* The command to execute is train_lora.sh, which needs the environment variable WEBUI_PORT, so I supplied that value.

* The command was to be executed at 8:43 am on the 7th of December, and 'day of week' does not matter, so there is a \*.
* After the \* is the complete path to my copy of the train_lora.sh script.  In addition to WEBUI_PORT, the script requires the name of the model to train, foobar in this case.
* If I did not have anything after foobar, I would not see any out put from the command. The greater than sign > means to write the output of the command to the file cron.log in my home directory.

Here is the contents of cron.log after the job was executed:

<img src='../data/cron-log.png'>

The script train_lora.sh, shown above in section 4, assembles the URL to execute, and then the line `echo $url` prints the assembled url. The last line is what the API returned.

## Next Semester

We will go through this notebook next semester.  I am sending it early in case you want to try things out over the break.

With any luck, d8ahazard will fix the bug in the dreambooth extension so that we can create models with the API. Right now we still need to create them with the GUI.

Being able to schedule training jobs for off hours should make it possible to start training more complicated models.